## Switch Direction 0/1 to Cardinal
* direction_id -> cardinal direction. i want to use stop_times_with_direction and just count, by route-id/direction-id how many stops in each stop_primary_direction group. then whatever is pluraltiy, assign that.
* you can take a look at the gtfs_analytics_catalog and find that file and just open it. the script to create it is in gtfs_funnel already, and you can see what that column means.
* https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/gtfs_analytics_data.yml
* https://github.com/cal-itp/data-analyses/blob/main/gtfs_funnel/stop_times_with_direction.py

### Amanda Questions
1. There isn't a `route_id` column. Which dataset do I merge in?
2. Do I group by `stop_id,stop_primary_direction, route_id` and total up all the unique `trip_instance_key`

In [1]:
import geopandas as gpd
import pandas as pd
from segment_speed_utils import helpers, time_series_utils
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### File

In [17]:
def load_most_current_date()->str:
    # from shared_utils import rt_utils
    dates_dictionary = rt_dates.DATES
    date_list = list(dates_dictionary.items())
    # Grab the last key-value pair
    last_key, last_value = date_list[-1]
    return last_value

In [18]:
current_date = load_most_current_date()

In [6]:
stop_times_file = GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction

In [7]:
stop_times_folder = GTFS_DATA_DICT.rt_vs_schedule_tables.dir

In [14]:
stop_times_path = f"{stop_times_folder}{stop_times_file}_{last_value}.parquet"

In [61]:
stop_times_df = gpd.read_parquet(stop_times_path)

In [20]:
stop_times_df.shape

(3558553, 14)

### By route-id/direction-id how many stops in each stop_primary_direction group. then whatever is pluraltiy, assign that.

In [62]:
stop_times_df.head(2)

,feed_key,stop_id,stop_sequence,schedule_gtfs_dataset_key,trip_instance_key,shape_array_key,stop_name,geometry,prior_stop_sequence,subseq_stop_sequence,stop_pair,stop_pair_name,stop_primary_direction,stop_meters
0,cad7d9dfbba521f1911ebdf004ad2007,16835,1,7cc0cb1871dfd558f11a2885c145d144,00006f4f87adb771f4a2557eae0a48ee,ae5bf985e5aa9097d4230efc815d0978,Vicente St & 30th Ave,POINT (-218853.130 -27989.188),<NA>,2,16835__13525,Vicente St & 30th Ave__29th Ave & Vicente St,Unknown,NaN
1,cad7d9dfbba521f1911ebdf004ad2007,13525,2,7cc0cb1871dfd558f11a2885c145d144,00006f4f87adb771f4a2557eae0a48ee,ae5bf985e5aa9097d4230efc815d0978,29th Ave & Vicente St,POINT (-218762.740 -27969.874),1,3,13525__16735,29th Ave & Vicente St__Ulloa St & 30th Ave,Eastbound,92.43


In [72]:
# stop_times_df.shape_array_key.value_counts()

In [73]:
stop_times_df.loc[stop_times_df.shape_array_key == "09992c708bfffb3d41246b99e33f787a"].explore('stop_primary_direction', marker_kwds = {'radius':10})

In [58]:
stop_times_agg = stop_times_df.groupby(['shape_array_key','stop_primary_direction']).agg({'trip_instance_key':'count'}).reset_index()

In [59]:
stop_times_agg.head(2)

,shape_array_key,stop_primary_direction,trip_instance_key
0,000bcf67803b18090cc3d61f40510310,Eastbound,2
1,000bcf67803b18090cc3d61f40510310,Southbound,2


In [60]:
stop_times_agg.loc[stop_times_agg.shape_array_key == "000bcf67803b18090cc3d61f40510310"]

,shape_array_key,stop_primary_direction,trip_instance_key
0,000bcf67803b18090cc3d61f40510310,Eastbound,2
1,000bcf67803b18090cc3d61f40510310,Southbound,2
2,000bcf67803b18090cc3d61f40510310,Unknown,1
3,000bcf67803b18090cc3d61f40510310,Westbound,1


In [51]:
stop_times_agg2 = stop_times_agg.sort_values(by = ['shape_array_key','stop_id','trip_instance_key'], ascending = [False, False, False])

In [53]:
stop_times_agg2.shape, stop_times_agg2.stop_id.nunique()

((204439, 4), 53205)

In [54]:
stop_times_agg2.stop_id.value_counts().describe()

count   53205.00
mean        3.84
std         4.98
min         1.00
25%         1.00
50%         2.00
75%         4.00
max       142.00
Name: stop_id, dtype: float64

In [55]:
stop_times_agg2.stop_id.value_counts().head()

52c2636c-34a3-434c-99ae-cdf3dc36d15c    142
e37ef534-cce7-4470-84b4-25ba98c51114    123
262                                      96
e204bb66-8ab9-4437-8d0d-2b603cdf0c43     93
ddf85f3f-f4ac-45e7-b439-1c31ed733ce1     70
Name: stop_id, dtype: int64

In [57]:
stop_times_agg2.loc[(stop_times_agg2.stop_id == "e204bb66-8ab9-4437-8d0d-2b603cdf0c43") & (stop_times_agg2.shape_array_key == "fffb34c9ca524fdb0bbb01b32505186f")]

,shape_array_key,stop_id,stop_primary_direction,trip_instance_key
204327,fffb34c9ca524fdb0bbb01b32505186f,e204bb66-8ab9-4437-8d0d-2b603cdf0c43,Eastbound,1


### Breakout `gtfs_funnel -> stop_times_with_direction`